In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%matplotlib inline


In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
       transforms.RandomResizedCrop(224),
       transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [5]:
num_epochs = 10
num_classes = 6
batch_size = 128
learning_rate = 0.001

In [6]:
 data_dir = '/content/drive/MyDrive/classification_dataset'


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),data_transforms[x]) for x in ['train']}

dataloaders =  {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train']}

class_names = image_datasets['train'].classes

print(dataset_sizes)
print(class_names)


image_testsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['test']}
testloaders = {x: torch.utils.data.DataLoader(image_testsets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['test']}


{'train': 543}
['bear', 'butterfly', 'camel', 'chimp', 'duck', 'elephant']


In [7]:
torch.cuda.is_available()

True

In [8]:


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
class ConvNet(nn.Module):
  def __init__(self, num_classes=6):
      super(ConvNet, self).__init__()
      self.layer1 = nn.Sequential(
          nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
          nn.BatchNorm2d(16),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2))
      self.layer2 = nn.Sequential(
          nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2))
      self.layer3 = nn.Sequential(
          nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2))
      self.layer4 = nn.Sequential(
          nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2))
      self.layer5 = nn.Sequential(
          nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2))
      self.fc = nn.Linear(7*7*256, num_classes)
    
      
  def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = self.layer3(out)
      out = self.layer4(out)
      out = self.layer5(out)
      out = out.reshape(out.size(0), -1)

      out = self.fc(out)
    
      return out

model = ConvNet(num_classes).to(device)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
total_step = len(dataloaders)
for epoch in range(num_epochs):
    for i,(images, classes) in enumerate(dataloaders['train']):
        images = images.to(device)
        labels = classes.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad() #sets dL/dw =0 
        loss.backward()       # computes dL/dw
        optimizer.step()      # w = w - lr*dL/dw
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [26]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, classes in testloaders['test']:
        images = images.to(device)
        labels = classes.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')
predicted = predicted.cpu().numpy()
labels = labels.cpu().numpy()

from sklearn import metrics
print(metrics.classification_report(labels,predicted))

Test Accuracy of the model on the test images: 52.5 %
              precision    recall  f1-score   support

           0       0.33      0.30      0.32        20
           1       0.52      0.75      0.61        20
           2       0.50      0.85      0.63        20
           3       0.59      0.65      0.62        20
           4       0.71      0.25      0.37        20
           5       0.70      0.35      0.47        20

    accuracy                           0.53       120
   macro avg       0.56      0.53      0.50       120
weighted avg       0.56      0.53      0.50       120

